# Public Company Financial Data

Author: Phuoc Le<br>
Google Colab Version: Version 2<br>
Data Source: Yahoo Finance

## Libraries Import

Import Python libraries needed for the notebook.  First we need to install some additional libraries.

In [2]:
!pip install yahoo-finance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130 kB 4.0 MB/s 
  Created wheel for yahoo-finance: filename=yahoo_finance-1.4.0-py3-none-any.whl size=7233 sha256=2ad75a993adda6314b6bdc9a094b80fbe3a9eb9e72fb706d0c1484aedb3aefed
  Stored in directory: /root/.cache/pip/wheels/74/3c/6e/504622cec9991612b9e3945fe8b375fef727854b42eb333c91
Successfully built yahoo-finance


In [3]:
!pip install requests_html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import yahoo_fin.stock_info as si
import pandas as pd
import plotly.express as px
from tqdm import tqdm

## Define List of Companies to Research

Enter list of stocks as a Python list

In [7]:
companies = ["AAPL", "MSFT", "WMT", "TGT", "MCD", "KO", "KMB", "RY"]

## Retrieve Financial Information for our Companies

1. The first thing to do is to retrieve the data.  Depending on how long the company list is, it may take a while.  We can't run Step 2 to clean up the data until this is complete, so be patient!  Wait for the indicator to get to 100% before proceeding.

In [8]:
# Get data in the current column for each stock's valuation table
# df1 contains basic company data
# df2 contains extra data

ticker_stats = {}
for ticker in companies:
    temp = si.get_stats_valuation(ticker)
    temp = temp.iloc[:,:2]
    temp.columns = ["Attribute", "Recent"]
 
    ticker_stats[ticker] = temp


# combine all the stats valuation tables into a single data frame
df1 = pd.concat(ticker_stats)
df1 = df1.reset_index()

del df1["level_1"]

# update column names
df1.columns = ["Ticker", "Attribute", "Recent"]

# Get extra stats
ticker_extra_stats = {}
for ticker in tqdm(companies):
    ticker_extra_stats[ticker] = si.get_stats(ticker)
    

df2 = pd.concat(ticker_extra_stats)

df2 = df2.reset_index()

del df2["level_1"]

df2.columns = ["Ticker", "Attribute", "Value"]

100%|██████████| 8/8 [00:06<00:00,  1.31it/s]


2. Now we do the data clean up, type conversions, and merging of the data into a final dataframe called **company_data** which we can now use.

In [9]:
# Convert from long dataframe to wide dataframe using the pivot() method.
df1_wide = df1.pivot(index = "Ticker", columns="Attribute", values="Recent")
df2_wide = df2.pivot(index = "Ticker", columns="Attribute", values="Value")

# Clean up column names to remove footnote indicators
df1_wide.rename(columns = {"Enterprise Value 3":"Enterprise Value"}, inplace = True)
df1_wide.rename(columns = {"Enterprise Value/EBITDA 6":"Enterprise Value/EBITDA"}, inplace = True)
df1_wide.rename(columns = {"Enterprise Value/Revenue 3":"Enterprise Value/Revenue"}, inplace = True)
df1_wide.rename(columns = {"Forward P/E 1":"Forward P/E"}, inplace = True)
df1_wide.rename(columns = {"Market Cap (intraday) 5":"Market Cap (intraday)"}, inplace = True)
df1_wide.rename(columns = {"PEG Ratio (5 yr expected) 1":"PEG Ratio (5 yr expected)"}, inplace = True)

df2_wide.rename(columns = {"% Held by Insiders 1":"% Held by Insiders"}, inplace = True)
df2_wide.rename(columns = {"% Held by Institutions 1":"% Held by Institutions"}, inplace = True)
df2_wide.rename(columns = {"200-Day Moving Average 3":"200-Day Moving Average"}, inplace = True)
df2_wide.rename(columns = {"5 Year Average Dividend Yield 4":"5 Year Average Dividend Yield %"}, inplace = True)
df2_wide.rename(columns = {"50-Day Moving Average 3":"50-Day Moving Average"}, inplace = True)
df2_wide.rename(columns = {"52 Week High 3":"52 Week High"}, inplace = True)
df2_wide.rename(columns = {"52 Week Low 3":"52 Week Low"}, inplace = True)
df2_wide.rename(columns = {"52-Week Change 3":"52-Week Change %"}, inplace = True)
df2_wide.rename(columns = {"Avg Vol (10 day) 3":"Avg Vol (10 day)"}, inplace = True)
df2_wide.rename(columns = {"Avg Vol (3 month) 3":"Avg Vol (3 month)"}, inplace = True)
df2_wide.rename(columns = {"Dividend Date 3":"Dividend Date"}, inplace = True)
df2_wide.rename(columns = {"Ex-Dividend Date 4":"Ex-Dividend Date"}, inplace = True)
df2_wide.rename(columns = {"Forward Annual Dividend Rate 4":"Forward Annual Dividend Rate"}, inplace = True)
df2_wide.rename(columns = {"Forward Annual Dividend Yield 4":"Forward Annual Dividend Yield %"}, inplace = True)
df2_wide.rename(columns = {"Last Split Date 3":"Last Split Date"}, inplace = True)
df2_wide.rename(columns = {"Last Split Factor 2":"Last Split Factor"}, inplace = True)
df2_wide.rename(columns = {"Operating Margin (ttm)":"Operating Margin (ttm) %"}, inplace = True)
df2_wide.rename(columns = {"Payout Ratio 4":"Payout Ratio %"}, inplace = True)
df2_wide.rename(columns = {"Profit Margin":"Profit Margin %"}, inplace = True)
df2_wide.rename(columns = {"Quarterly Earnings Growth (yoy)":"Quarterly Earnings Growth (yoy) %"}, inplace = True)
df2_wide.rename(columns = {"Quarterly Revenue Growth (yoy)":"Quarterly Revenue Growth (yoy) %"}, inplace = True)
df2_wide.rename(columns = {"Return on Assets (ttm)":"Return on Assets (ttm) %"}, inplace = True)
df2_wide.rename(columns = {"Return on Equity (ttm)":"Return on Equity (ttm) %"}, inplace = True)
df2_wide.rename(columns = {"S&P500 52-Week Change 3":"S&P500 52-Week Change %"}, inplace = True)
df2_wide.rename(columns = {"Shares Outstanding 5":"Shares Outstanding"}, inplace = True)
df2_wide.rename(columns = {"Trailing Annual Dividend Rate 3":"Trailing Annual Dividend Rate"}, inplace = True)
df2_wide.rename(columns = {"Trailing Annual Dividend Yield 3":"Trailing Annual Dividend Yield %"}, inplace = True)

# Convert string values to float as necessary
df1_wide['Trailing P/E'] = df1_wide['Trailing P/E'].astype(float)
df1_wide['Enterprise Value/EBITDA'] = df1_wide['Enterprise Value/EBITDA'].astype(float)
df1_wide['Enterprise Value/Revenue'] = df1_wide['Enterprise Value/Revenue'].astype(float)
df1_wide['Forward P/E'] = df1_wide['Forward P/E'].astype(float)
df1_wide['PEG Ratio (5 yr expected)'] = df1_wide['PEG Ratio (5 yr expected)'].astype(float)
df1_wide['Price/Book (mrq)'] = df1_wide['Price/Book (mrq)'].astype(float)
df1_wide['Price/Sales (ttm)'] = df1_wide['Price/Sales (ttm)'].astype(float)

df2_wide['% Held by Insiders'] = df2_wide['% Held by Insiders'].str.strip('%').astype('float') / 100.0
df2_wide['% Held by Institutions'] = df2_wide['% Held by Institutions'].str.strip('%').astype('float') / 100.0
df2_wide['5 Year Average Dividend Yield %'] = df2_wide['5 Year Average Dividend Yield %'].astype(float) / 100.0
df2_wide['50-Day Moving Average'] = df2_wide['50-Day Moving Average'].astype(float)
df2_wide['52 Week High'] = df2_wide['52 Week High'].astype(float)
df2_wide['52 Week Low'] = df2_wide['52 Week Low'].astype(float)
df2_wide['52-Week Change %'] = df2_wide['52-Week Change %'].str.strip('%').astype('float') / 100.0
df2_wide['Beta (5Y Monthly)'] = df2_wide['Beta (5Y Monthly)'].astype(float)
df2_wide['Book Value Per Share (mrq)'] = df2_wide['Book Value Per Share (mrq)'].astype(float)
df2_wide['Current Ratio (mrq)'] = df2_wide['Current Ratio (mrq)'].astype(float)
df2_wide['Diluted EPS (ttm)'] = df2_wide['Diluted EPS (ttm)'].astype(float)
df2_wide['Forward Annual Dividend Rate'] = df2_wide['Forward Annual Dividend Rate'].astype(float)
df2_wide['Forward Annual Dividend Yield %'] = df2_wide['Forward Annual Dividend Yield %'].str.strip('%').astype('float') / 100.0
df2_wide['Operating Margin (ttm) %'] = df2_wide['Operating Margin (ttm) %'].str.strip('%').astype('float') / 100.0
df2_wide['Payout Ratio %'] = df2_wide['Payout Ratio %'].str.strip('%').astype('float') / 100.0
df2_wide['Profit Margin %'] = df2_wide['Profit Margin %'].str.strip('%').astype('float') / 100.0
df2_wide['Quarterly Earnings Growth (yoy) %'] = df2_wide['Quarterly Earnings Growth (yoy) %'].str.strip('%').astype('float') / 100.0
df2_wide['Quarterly Revenue Growth (yoy) %'] = df2_wide['Quarterly Revenue Growth (yoy) %'].str.strip('%').astype('float') / 100.0
df2_wide['Return on Assets (ttm) %'] = df2_wide['Return on Assets (ttm) %'].str.strip('%').astype('float') / 100.0
df2_wide['Return on Equity (ttm) %'] = df2_wide['Return on Equity (ttm) %'].str.strip('%').astype('float') / 100.0
df2_wide['Revenue Per Share (ttm)'] = df2_wide['Revenue Per Share (ttm)'].astype(float)
df2_wide['S&P500 52-Week Change %'] = df2_wide['S&P500 52-Week Change %'].str.strip('%').astype('float') / 100.0
df2_wide['Total Cash Per Share (mrq)'] = df2_wide['Total Cash Per Share (mrq)'].astype(float)
df2_wide['Total Debt/Equity (mrq)'] = df2_wide['Total Debt/Equity (mrq)'].astype(float)
df2_wide['Trailing Annual Dividend Rate'] = df2_wide['Trailing Annual Dividend Rate'].astype(float)
df2_wide['Trailing Annual Dividend Yield %'] = df2_wide['Trailing Annual Dividend Yield %'].str.strip('%').astype('float') / 100.0
df2_wide['200-Day Moving Average'] = df2_wide['200-Day Moving Average'].astype(float)

# Merge the dataframes into company_data
company_data = pd.concat([df1_wide, df2_wide], axis=1, join='inner')

Here we display the dataframe's content so we know what we're working with:

In [10]:
company_data.head(10)

Attribute,Enterprise Value,Enterprise Value/EBITDA,Enterprise Value/Revenue,Forward P/E,Market Cap (intraday),PEG Ratio (5 yr expected),Price/Book (mrq),Price/Sales (ttm),Trailing P/E,% Held by Insiders,...,"Shares Short (prior month Apr 13, 2022) 4","Short % of Float (May 12, 2022) 4","Short % of Shares Outstanding (May 12, 2022) 4","Short Ratio (May 12, 2022) 4",Total Cash (mrq),Total Cash Per Share (mrq),Total Debt (mrq),Total Debt/Equity (mrq),Trailing Annual Dividend Rate,Trailing Annual Dividend Yield %
Ticker,,,,,,,,,,,,,,,,,,,,,
AAPL,2.29T,17.23,5.94,22.57,2.22T,2.49,32.98,5.90,22.33,0.0007,...,103.29M,0.67%,0.67%,1.03,51.51B,3.18,119.98B,178.02,0.88,0.0063
KMB,52.06B,16.41,2.63,22.37,43.48B,2.88,61.67,2.21,24.91,0.0016,...,8.98M,2.28%,2.28%,3.2,493M,1.46,9.33B,1092.51,4.58,0.0349
KO,295.70B,18.64,7.37,24.63,264.35B,3.38,10.64,6.61,25.73,0.0066,...,28.08M,0.73%,0.72%,1.69,10.36B,2.39,42.14B,157.01,1.70,0.0265
MCD,214.57B,18.21,9.03,23.15,169.36B,3.52,NaN,7.24,24.18,0.0016,...,5.95M,0.83%,0.83%,2.22,2.36B,3.19,47.59B,NaN,5.34,0.0219
MSFT,1.85T,18.72,9.58,23.42,1.89T,1.70,11.59,9.91,26.36,0.0008,...,35.58M,0.51%,0.51%,1.07,104.66B,13.99,77.98B,47.86,2.42,0.0092
RY,NaN,NaN,NaN,11.19,139.31B,NaN,1.89,3.88,11.18,0.0001,...,7.23M,NaN,0.89%,11.78,683.12B,482.21,363.67B,NaN,4.44,0.0442
TGT,85.41B,7.14,0.81,11.07,74.93B,1.24,5.84,0.75,11.46,0.0022,...,10.49M,1.81%,1.80%,2.45,1.11B,2.40,17.32B,160.78,3.38,0.0216
WMT,370.34B,12.22,0.64,17.76,327.77B,2.63,3.94,0.58,24.45,0.4770,...,13.33M,0.90%,0.47%,1.8,11.82B,4.29,66.82B,77.82,1.65,0.0134


## Plot basic valuation data to find value investments

In [11]:
# Plot out the data
chart1 = px.scatter(company_data,
                   x='Quarterly Revenue Growth (yoy) %',
                   y='Price/Sales (ttm)',
                   size = "Trailing P/E",
                   hover_name = company_data.index,
                   title="Quarterly Revenue Growth (yoy) % vs. Price/Sales (ttm) vs. Trailing P/E",
                   width=1250,
                   height=700)

# Update chart layouts
chart1.update_layout(
    margin=dict(l=20, r=20, t=40, b=20),
    paper_bgcolor="#ffffff",
)

chart1.show()

## Plot dividend data to research dividend stocks

In [12]:
# Plot out the data
chart1 = px.scatter(company_data,
                   x='Payout Ratio %',
                   y='Trailing Annual Dividend Yield %',
                   size = "5 Year Average Dividend Yield %",
                   hover_name = company_data.index,
                   title="Payout Ratio %' vs. Trailing Annual Dividend Yield % vs. 5 Year Average Dividend Yield %",
                   width=900,
                   height=500)

# Update chart layouts
chart1.update_layout(
    margin=dict(l=20, r=20, t=40, b=20),
    paper_bgcolor="#eeeeee",
)

chart1.show()

# Retrieve Financial Statements for our Companies

We'll start by pulling the Balance Sheet and then move on to the Income Statement & Cash Flow Statement.

## Balance Sheet

We'll start by getting the Balance Sheet for one company.  That's done like this:

In [13]:
balance_sheet = si.get_balance_sheet("aapl")

display(balance_sheet)

endDate,2021-09-25,2020-09-26,2019-09-28,2018-09-29
Breakdown,,,,
totalLiab,287912000000,258549000000,248028000000,258578000000
totalStockholderEquity,63090000000,65339000000,90488000000,107147000000
otherCurrentLiab,53577000000,47867000000,43242000000,39293000000
totalAssets,351002000000,323888000000,338516000000,365725000000
commonStock,57365000000,50779000000,45174000000,40201000000
otherCurrentAssets,14111000000,11264000000,12352000000,12087000000
retainedEarnings,5562000000,14966000000,45898000000,70400000000
otherLiab,43050000000,46108000000,50503000000,48914000000
treasuryStock,163000000,-406000000,-584000000,-3454000000


But what's more interesting is to get the Balance Sheets for **<u>all</u>** the companies in our list.  That way we can combine them and do interesting things with them.  That's done with the following code:

In [14]:
balance_sheets = {}
for ticker in companies:
    balance_sheets[ticker] = si.get_balance_sheet(ticker)

Now that all the balance sheets are combined, we can look at values from the balance sheets across multiple companies at once.  Since each individual balance sheet may have different column headers (from different dates), we’ll just get the most recent column of data from the balance sheet for each stock.  This code combines the most recent columns from the balance sheets of all companies into one dataframe.

In [15]:
recent_sheets = {ticker : sheet.iloc[:,:1] for ticker,sheet in balance_sheets.items()}

for ticker in recent_sheets.keys():
    recent_sheets[ticker].columns = ["Recent"]

# combine all balance sheets together
combined_sheets = pd.concat(recent_sheets)

# reset index to pull in ticker
combined_sheets = combined_sheets.reset_index()

# update column names
combined_sheets.columns = ["Ticker", "Breakdown", "Recent"]

combined_sheets

,Ticker,Breakdown,Recent
0,AAPL,totalLiab,2.879120e+11
1,AAPL,totalStockholderEquity,6.309000e+10
2,AAPL,otherCurrentLiab,5.357700e+10
3,AAPL,totalAssets,3.510020e+11
4,AAPL,commonStock,5.736500e+10
...,...,...,...
203,RY,netTangibleAssets,7.665800e+10
204,RY,shortTermInvestments,3.875410e+11
205,RY,netReceivables,1.334200e+10
206,RY,longTermDebt,1.015120e+11


From here, we can pull any data from the balance sheets of all the companies. For example, we can look at the Total Assets for each stock like this:

In [16]:
combined_sheets[combined_sheets.Breakdown == "longTermDebt"]

,Ticker,Breakdown,Recent
20,AAPL,longTermDebt,1.091060e+11
46,MSFT,longTermDebt,5.007400e+10
72,WMT,longTermDebt,3.595900e+10
98,TGT,longTermDebt,1.158200e+10
124,MCD,longTermDebt,3.562680e+10
153,KO,longTermDebt,3.813000e+10
179,KMB,longTermDebt,8.107000e+09
206,RY,longTermDebt,1.015120e+11


## Income Statements

Income statements can be downloaded from Yahoo Finance using the get_income_statement method:

In [17]:
income = si.get_income_statement("aapl")

display(income)

endDate,2021-09-25,2020-09-26,2019-09-28,2018-09-29
Breakdown,,,,
researchDevelopment,21914000000,18752000000,16217000000,14236000000
effectOfAccountingCharges,None,None,None,None
incomeBeforeTax,109207000000,67091000000,65737000000,72903000000
minorityInterest,None,None,None,None
netIncome,94680000000,57411000000,55256000000,59531000000
sellingGeneralAdministrative,21973000000,19916000000,18245000000,16705000000
grossProfit,152836000000,104956000000,98392000000,101839000000
ebit,108949000000,66288000000,63930000000,70898000000
operatingIncome,108949000000,66288000000,63930000000,70898000000


Using the income statement, we can examine specific values, such as total revenue, gross profit, total expenses, etc.

To get the **total revenue**, we just need to apply a filter like previously.  Other items like **grossProfit**, **operatingIncome**, etc. works the same.

In [18]:
income.loc["totalRevenue"]

endDate
2021-09-25    365817000000
2020-09-26    274515000000
2019-09-28    260174000000
2018-09-29    265595000000
Name: totalRevenue, dtype: object

## Getting the income statement from each company in our list

Next, let’s retrieve the income statement for each company in our **companies** list.

In [19]:
income_statements = {}
for ticker in companies:
    income_statements[ticker] = si.get_income_statement(ticker)

Now, we can look at metrics in the income statement across multiple companies at once. First, we just need to combine the income statements together, similar to how we combined the balance sheets above.

In [20]:
recent_income_statements = {ticker : sheet.iloc[:,:1] for ticker,sheet in income_statements.items()}

for ticker in recent_income_statements.keys():
    recent_income_statements[ticker].columns = ["Recent"]

combined_income = pd.concat(recent_income_statements)

combined_income = combined_income.reset_index()

combined_income.columns = ["Ticker", "Breakdown", "Recent"]

Now that we have a combined view of the income statements across companies, we can examine specific values in the income statements, such as Total Revenue, for example.

In [21]:
combined_income[combined_income.Breakdown == "totalRevenue"]

,Ticker,Breakdown,Recent
15,AAPL,totalRevenue,365817000000
37,MSFT,totalRevenue,168088000000
59,WMT,totalRevenue,572754000000
81,TGT,totalRevenue,106005000000
103,MCD,totalRevenue,23222900000
125,KO,totalRevenue,38655000000
147,KMB,totalRevenue,19440000000
169,RY,totalRevenue,50446000000


In [22]:
combined_income

,Ticker,Breakdown,Recent
0,AAPL,researchDevelopment,21914000000
1,AAPL,effectOfAccountingCharges,None
2,AAPL,incomeBeforeTax,109207000000
3,AAPL,minorityInterest,None
4,AAPL,netIncome,94680000000
...,...,...,...
171,RY,costOfRevenue,0
172,RY,totalOtherIncomeExpenseNet,-1287000000
173,RY,discontinuedOperations,None
174,RY,netIncomeFromContinuingOps,16050000000


## Cash Flow Statement

In this section, we’ll extract cash flow statements. We can do that using the get_cash_flow method.

In [23]:
flow = si.get_cash_flow("aapl")

Here’s the first few rows of the cash flow statement:

In [24]:
flow.head(20)

endDate,2021-09-25,2020-09-26,2019-09-28,2018-09-29
Breakdown,,,,
investments,-2819000000,5335000000,58093000000,30845000000
changeToLiabilities,14002000000,-1981000000,-2548000000,9172000000
totalCashflowsFromInvestingActivities,-14545000000,-4289000000,45896000000,16066000000
netBorrowings,12665000000,2499000000,-7819000000,432000000
totalCashFromFinancingActivities,-93353000000,-86820000000,-90976000000,-87876000000
changeToOperatingActivities,-6146000000,881000000,-896000000,30016000000
issuanceOfStock,1105000000,880000000,781000000,669000000
netIncome,94680000000,57411000000,55256000000,59531000000
changeInCash,-3860000000,-10435000000,24311000000,5624000000


Now let’s get the cash flow statements of each company on our **companies** list.

In [25]:
cash_flows = {}
for ticker in companies:
    cash_flows[ticker] = si.get_cash_flow(ticker)

Again, we combine the datasets above, using similar code as before.

In [26]:
recent_cash_flows = {ticker : flow.iloc[:,:1] for ticker,flow in cash_flows.items()}


for ticker in recent_cash_flows.keys():
    recent_cash_flows[ticker].columns = ["Recent"]


combined_cash_flows = pd.concat(recent_cash_flows)

combined_cash_flows = combined_cash_flows.reset_index()

combined_cash_flows.columns = ["Ticker", "Breakdown", "Recent"]

Now, we can examine information in the cash flow statements across all the stocks in our list.

## Getting dividends paid across companies

One example to look at in a cash flow statement is the amount of dividends paid, which we can see across the companies in our list by using the filter below.

In [27]:
combined_cash_flows[combined_cash_flows.Breakdown == "dividendsPaid"]

,Ticker,Breakdown,Recent
13,AAPL,dividendsPaid,-14467000000
33,MSFT,dividendsPaid,-16521000000
51,WMT,dividendsPaid,-6152000000
69,TGT,dividendsPaid,-1548000000
86,MCD,dividendsPaid,-3918600000
106,KO,dividendsPaid,-7252000000
126,KMB,dividendsPaid,-1516000000
145,RY,dividendsPaid,-6420000000


## Getting stock issuance information

Here’s another example – this time, we’ll look at debt-related numbers across the cash flow statements.

In [28]:
combined_cash_flows[combined_cash_flows.Breakdown == "issuanceOfStock"]

,Ticker,Breakdown,Recent
6,AAPL,issuanceOfStock,1105000000
25,MSFT,issuanceOfStock,1693000000
63,TGT,issuanceOfStock,8000000
78,MCD,issuanceOfStock,285700000
98,KO,issuanceOfStock,702000000
118,KMB,issuanceOfStock,65000000
138,RY,issuanceOfStock,4853000000


## Reference

Reference Information

In [34]:
# Print dataframe's columns to a list

#print(company_extra_stats_wide.columns.tolist())

In [35]:
# Pull out just Trailing P/E to another dataframe
#trailing_pe = company_stats[company_stats["Attribute"].str.contains("Trailing")]

#trailing_pe

In [31]:
# S&P 500 List
sp500_list = si.tickers_sp500()

print(sp500_list)

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BALL', 'BAX', 'BBWI', 'BBY', 'BDX', 'BEN', 'BF-B', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BMY', 'BR', 'BRK-B', 'BRO', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CEG', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CPT', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTRA', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISH

In [32]:
# Dow List
dow_list = si.tickers_dow()

print(dow_list)

['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT']


In [33]:
# NASDAQ List
nasdaq_list = si.tickers_nasdaq()

print(nasdaq_list)

['AACG', 'AACI', 'AACIU', 'AACIW', 'AADI', 'AADR', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPL', 'AATC', 'AAWW', 'AAXJ', 'ABCB', 'ABCL', 'ABCM', 'ABEO', 'ABGI', 'ABIO', 'ABMD', 'ABNB', 'ABOS', 'ABSI', 'ABST', 'ABTX', 'ABUS', 'ABVC', 'ACAB', 'ACABU', 'ACABW', 'ACAD', 'ACAH', 'ACAHU', 'ACAHW', 'ACAX', 'ACAXR', 'ACAXU', 'ACAXW', 'ACB', 'ACBA', 'ACBAU', 'ACBAW', 'ACCD', 'ACER', 'ACET', 'ACEV', 'ACEVU', 'ACEVW', 'ACGL', 'ACGLN', 'ACGLO', 'ACHC', 'ACHL', 'ACHV', 'ACIU', 'ACIW', 'ACKIT', 'ACKIU', 'ACKIW', 'ACLS', 'ACLX', 'ACMR', 'ACNB', 'ACON', 'ACONW', 'ACOR', 'ACQR', 'ACQRU', 'ACQRW', 'ACRS', 'ACRX', 'ACST', 'ACT', 'ACTD', 'ACTDU', 'ACTDW', 'ACTG', 'ACVA', 'ACWI', 'ACWX', 'ACXP', 'ADAG', 'ADAL', 'ADALU', 'ADALW', 'ADAP', 'ADBE', 'ADER', 'ADERU', 'ADERW', 'ADES', 'ADGI', 'ADI', 'ADIL', 'ADILW', 'ADMA', 'ADMP', 'ADN', 'ADNWW', 'ADOC', 'ADOCR', 'ADOCW', 'ADP', 'ADPT', 'ADRE', 'ADSE', 'ADSEW', 'ADSK', 'ADTH', 'ADTHW', 'ADTN', 'ADTX', 'ADUS', 'ADV', 'ADVM', 'ADVWW', 'ADXN', 'AEAC', 'AEACU', 'AEACW', 'A